# Projeto - Cálculo de raio de um asteroide

#### Amanda Lucio, Nayara Gomes

O objetivo dos modelos é medir o diâmetro de um asteroide a partir de medidas sobre o mesmo. O *Dataset* utilizado será  **"Open Asteroid Dataset"** retirado do site https://www.kaggle.com.

Após baixar no Kaggle o arquivo *Asteroid_Updated.csv* contendo os dados sobre asteroides, vamos preparar nosso ambiente com as bibliotecas necessárias e depois importaremos os dados!

## Importando as bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

%matplotlib inline

In [ ]:
dataAsteroid = pd.read_csv('Asteroid.csv')
dataAsteroid.head()

### Variáveis do *Dataset*

In [ ]:
dataAsteroid.columns

### Regressão Linear

Para treinar o modelo de regressão, primeiro precisaremos dividir nossos dados em uma matriz **x** que contenha os dados das variáveis preditoras e uma matriz **y** com os dados da variável de destino, nesse caso a coluna diameter.